In [1]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random, pickle
import Levenshtein as lev
from urllib.request import urlopen
import urllib.parse, os
import numpy as np
from IPython.display import display

In [94]:
CT_EV_raw = pd.read_csv('Figures/ct_ev_registrations_public.csv', index_col=0)
NJ_EV_raw = pd.read_csv('Figures/nj_ev_registrations_public.csv', index_col=0)
NY_EV_raw = pd.read_csv('Figures/ny_ev_registrations_public.csv', index_col=0)
MD_EV_raw = pd.read_csv('Figures/md_ev_registrations_public.csv', index_col=0)

/opt/anaconda3/envs/metis/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
alt_fuel_stations_ny = pd.read_csv('Figures/alt_fuel_stations_ny.csv', index_col=0)
alt_fuel_stations_nj = pd.read_csv('Figures/alt_fuel_stations_nj.csv', index_col=0)
alt_fuel_stations_ct = pd.read_csv('Figures/alt_fuel_stations_ct.csv', index_col=0)
alt_fuel_stations_md = pd.read_csv('Figures/alt_fuel_stations_md.csv', index_col=0)

In [25]:
ny_stations_count = alt_fuel_stations_ny.ZIP.value_counts()
nj_stations_count = alt_fuel_stations_nj.ZIP.value_counts()
ct_stations_count = alt_fuel_stations_ct.ZIP.value_counts()
md_stations_count = alt_fuel_stations_md.ZIP.value_counts()

In [197]:
md_stations_count

21201    47
21044    44
20850    40
21202    34
20852    30
         ..
21903     1
21769     1
21766     1
21619     1
21778     1
Name: ZIP, Length: 233, dtype: int64

In [152]:
stations_count = ny_stations_count.append(nj_stations_count).append(ct_stations_count).append(md_stations_count)
stations_count.name = 'num_stations'

In [196]:
stations_count.index = stations_count.index.astype('str').map(fill_zip)

### Process MD electric vehicle data.
Below data show all vehicles registered that year. That is, an owner has bought a vehicle and registered it with the state. Not that it's a tally of all vehicles "on the road" in the state.

In [48]:
MD_EV_raw.index = pd.to_datetime(MD_EV_raw.index)
MD_EV_raw['year'] = MD_EV_raw.index.year
MD_EV_count = MD_EV_raw[(MD_EV_raw.year == 2021) & (MD_EV_raw.Zip_Code.str[0] == '2')][['Zip_Code', 'year', 'Count']]
MD_EV_count = MD_EV_count.groupby('Zip_Code').Count.sum()
MD_EV_count

Zip_Code
20601     775
20602     908
20603    1631
20606      12
20607    1055
         ... 
21918      36
21919     100
21921    1006
21930       4
23703       1
Name: Count, Length: 432, dtype: int64

In [195]:
def fill_zip(x):
    x = str(x)
    if len(x) == 4:
        x = '0' + x
    return x

In [78]:
NJ_EV_raw = pd.read_csv('Figures/nj_ev_registrations_public.csv', index_col=0)
NJ_EV_raw['Registration Valid Date'] = pd.to_datetime(NJ_EV_raw['Registration Valid Date'])
NJ_EV_raw['reg_year'] = NJ_EV_raw['Registration Valid Date'].dt.year
NJ_EV_raw['ZIP Code'] = NJ_EV_raw['ZIP Code'].astype('str')
NJ_EV_raw['ZIP Code'] = NJ_EV_raw['ZIP Code'].apply(lambda x: fill_zip(x))
NJ_EV_count = NJ_EV_raw[NJ_EV_raw.reg_year == 2020]['ZIP Code'].value_counts()
NJ_EV_count = NJ_EV_count[(NJ_EV_count.index.str[1] == '7' )| (NJ_EV_count.index.str[1] == '8')]

In [79]:
NJ_EV_count

08540    1810
07677    1165
08831    1098
08002     994
08820     928
         ... 
07308       1
07851       1
07061       1
07428       1
08321       1
Name: ZIP Code, Length: 616, dtype: int64

In [86]:
NY_EV_raw['Registration Expiration Date'] = pd.to_datetime(NY_EV_raw['Registration Expiration Date'])
NY_EV_count = NY_EV_raw[NY_EV_raw['Registration Expiration Date'].dt.year == 2020].index.value_counts()
NY_EV_count = NY_EV_count[NY_EV_count.index.astype('str').str[0] == '1']

In [87]:
NY_EV_count

10583    6025
11746    5798
14850    4510
11791    4092
11743    3991
         ... 
15241       1
15001       1
19720       1
18104       1
18801       1
Name: ZIP Code, Length: 1457, dtype: int64

In [104]:
CT_EV_raw = pd.read_csv('Figures/ct_ev_registrations_public.csv', index_col=0)
CT_EV_raw['Registration Valid Date'] = pd.to_datetime(CT_EV_raw['Registration Valid Date'])
CT_EV_raw['ZIP Code'] = CT_EV_raw['ZIP Code'].astype('str').apply(lambda x: fill_zip(x))
CT_EV_count = CT_EV_raw[CT_EV_raw['Registration Valid Date'].dt.year == 2018]['ZIP Code'].value_counts()
CT_EV_count

06880    266
06830    226
06840    152
06820    152
06831    150
        ... 
06079      1
06021      1
06724      1
02184      1
06490      1
Name: ZIP Code, Length: 272, dtype: int64

In [109]:
EV_count = CT_EV_count.append(NJ_EV_count).append(NY_EV_count).append(MD_EV_count)
EV_count

06880     266
06830     226
06840     152
06820     152
06831     150
         ... 
21918      36
21919     100
21921    1006
21930       4
23703       1
Length: 2777, dtype: int64

In [112]:
EV_count_df = pd.DataFrame(EV_count, columns = ['reg_vehicles'])
EV_count_df.index.name = 'zipcode'

In [146]:
EV_count_df

,reg_vehicles
zipcode,
06880,266
06830,226
06840,152
06820,152
06831,150
...,...
21918,36
21919,100
21921,1006


In [219]:
EV_count_df.index = EV_count_df.index.map(fill_zip)

In [227]:
merged_EV_stations_df = EV_count_df.merge(stations_count, left_index=True, right_index=True, how='left')
# merged_EV_stations_df.dropna(inplace=True)

In [228]:
merged_EV_stations_df.index.name='zipcode'

In [230]:
merged_EV_stations_df.fillna(float(0), inplace=True)

In [232]:
merged_EV_stations_df.sort_values('num_stations', ascending=False)

,reg_vehicles,num_stations
zipcode,,
12866,2248,72.0
21201,1113,47.0
21044,5898,44.0
12205,826,40.0
20850,10523,40.0
...,...,...
11365,905,0.0
11363,353,0.0
11362,855,0.0


In [233]:
merged_EV_stations_df.to_pickle('Figures/merged_EV_stations_df')

In [235]:
NY_EV_count.loc[11213]

195